In [1]:
!pip install folium haversine
import sys,os
sys.path.append('/ikaro/PycharmProjects/medium/scripts')
import numpy as np
from dataclasses import dataclass,field
import pandas as pd
from searchrescue import show_map, random_walk_simple,random_walk_with_elevation, get_elevation_grid

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
@dataclass
class LostHiker:
    start_long: float
    start_lat: float
    max_speed: float  #Maximum speed in meters per second
    hours_missing : float
    missing_radius: str = field(init=False, repr=True)
    loc: tuple  = field(init=False, repr=True)
        
    def __post_init__(self):
        self.missing_radius = self.max_speed*self.hours_missing*3600 
        self.loc=(self.start_long,self.start_lat) #Short hand for cleaner code

hiker=LostHiker(start_long=37.718780413149968,start_lat=-119.6403873442959697,max_speed=0.5,hours_missing=24)
show_map(hiker,plot_origin=True)

In [3]:
show_map(hiker,add_missing_radius=True,zoom=11,plot_origin=True)

In [4]:
trails=random_walk_simple(hiker)
show_map(hiker,trail=trails[0],add_missing_radius=True,zoom=11,plot_origin=True)

In [ ]:
grid,xy_grid=get_elevation_grid(hiker,grid_range=6000,grid_resolution=10,recache=True)
show_map(hiker,trail=None, add_missing_radius=False, zoom=14, sims=None, trail_points=False,trail_heat=False, grid=grid, probs=None, trail_enumerate=False, plot_origin=True, probs_point=True)

Loading cache from local diretory /ikaro
Loading cache file elevation_res10m_v3.csv
Total latitude points in the grid =600
Total longitude points in the grid =600
df.shape=(360600, 25)
 0.560 valid elevation data (N=360600)
Querying elevation data for grid size=360600 50-batch queries=3606 grid_N=600 (grid_N^2= 360000 ) 
Getting 2013.0 /3606 next batch
Got 0.560 valid elevation data (N=32) from query 2013.0
Getting 2015.0 /3606 next batch
Got 0.560 valid elevation data (N=65) from query 2015.0
Getting 2016.0 /3606 next batch
Got 0.560 valid elevation data (N=5) from query 2016.0
Getting 2017.0 /3606 next batch
Got 0.560 valid elevation data (N=4) from query 2017.0
Getting 2018.0 /3606 next batch
Got 0.560 valid elevation data (N=36) from query 2018.0
Getting 2019.0 /3606 next batch
Got 0.560 valid elevation data (N=7) from query 2019.0
Getting 2020.0 /3606 next batch
Got 0.560 valid elevation data (N=12) from query 2020.0
Getting 2021.0 /3606 next batch
Got 0.561 valid elevation data (

In [ ]:
def random_walk_with_elevation2(hiker, N=1, end_point_only=True, xy_grid=None,verbose=False):
    '''
        Assumptions:
        1. Velocity is a function of elevation
        2. Hiker cannot go beyond a positive elevation limit (will try another direction before yielding an iteration)
        3. If Hiker goes beyond a negative elevation limit it will stay there (terminal point, simulating a fall)
    '''
    distance5min = hiker.max_speed * 60 * 5  # Speed in m/5 minutes
    trails = []
    max_grad = 10 * 2 * np.pi / 180  # Assuming max 45 deg climbing gradient
    fall_grad = -30 * 2 * np.pi / 180  # Assuming max 45 deg climbing gradient
    maxg = 0
    for n in range(N):
        trail = [hiker.loc]
        terminate = False
        t = 0
        while t < (12 * hiker.hours_missing):
            trail.append(new_loc)
            t += 1
            if terminate:
                break
        if end_point_only:
            trails.append(trail[-1])
        else:
            trails.append(trail)
    return trails

In [ ]:
#xy_grid
long_map=grid[['longitude','east_offset']].drop_duplicates()
lat_map=grid[['latitude','south_offset']].drop_duplicates()
#print(grid.south_offset.nunique())
#print(grid.east_offset.nunique())
print(grid.latitude.unique())
#print(grid.longitude.nunique())

In [ ]:
trails=random_walk_with_elevation(hiker,N=1000,end_point_only=True,xy_grid=xy_grid,grid=grid)
data=pd.DataFrame(np.squeeze(np.array(trails)),columns=['latitude','longitude'])
print(data)
os.sys.exit()
#show_map(hiker,add_missing_radius=True,zoom=12,sims=data,trail_points=False)

In [ ]:
#df_elevation=get_elevation_grid(hiker.loc)
#df_elevation=pd.read_csv('elevation.csv')
#df_elevation=df_elevation.dropna(subset='elev_meters')
#df_elevation['point'] = [(x, y) for x,y in zip(df_elevation['latitude'],df_elevation['longitude'])]
center = grid.loc[np.argmin((hiker.loc[0]-grid['latitude'])**2 + (hiker.loc[1]-grid['longitude'])**2)]
center_elev=xy_grid.loc[center.south_offset,center.east_offset]
#Check if elevation is within hiker limits, if so, continue path

center = grid.loc[np.argmin((hiker.loc[0]-grid['latitude'])**2 + (hiker.loc[1]-grid['longitude'])**2)]
center_elev=xy_grid.loc[center.south_offset,center.east_offset]
#Check if elevation is within hiker limits, if so, continue path

In [ ]:
trails=radom_walk_simple(hiker,N=1000,end_point_only=True)
sims=pd.DataFrame(np.squeeze(np.array(trails)),columns=['latitude','longitude'])
show_map(hiker,add_missing_radius=True,zoom=15,sims=data,trail_points=True)

In [ ]:
show_map(hiker,add_missing_radius=True,zoom=15,sims=data,trail_points=True)

## Define Rescue Efforts

### Create Rescue Grid & Probability Prior from simulations

In [ ]:
rescue_grid=get_rescue_grid(hiker,grid_range=3500,grid_resolution=100,simulation_data=data)

In [ ]:
show_map(hiker,probs=rescue_grid)#,sims=data,trail_points=True,trail_heat=False)

### Create a rescue plan, evaluate probability of sucess, and update probability posterior

In [ ]:
# Define rescure properties
@dataclass
class Rescuer:
    max_grid_points: int  #Number of grid points the rescuer can traverse
    prob_find: float #Probability of find hiker if the hiker is in the same grid point as the rescuer
    
rescuer=Rescuer(max_grid_points=15,prob_find=0.999)
rescue_plan,p=greedy_search(rescuer,rescue_grid,sims=data)
print("Probabiliy of rescue=%0.3f"%p)
show_map(hiker,trail=rescue_plan,probs=rescue_grid,trail_enumerate=True,plot_origin=False,zoom=15)


In [ ]:
rescue_grid_post=get_post_rescue_grid(rescue_grid, rescue_plan)
show_map(hiker,probs=rescue_grid_post,probs_point=False)

In [ ]:
attempts=2000
rescue_grid_post=rescue_grid.copy()
start_at=rescue_grid.iloc[rescue_grid['prob'].idxmax()].copy().to_frame().T.reset_index(drop=True)
for i in range(attempts):
    rescue_plan,p=greedy_search(rescuer,rescue_grid_post,sims=data,start_at=start_at)
    if not i%500:
        print("%s: Probabiliy of rescue=%0.3f"%(i,p))
    rescue_grid_post=get_post_rescue_grid(rescue_grid_post, rescue_plan)

show_map(hiker,probs=rescue_grid_post,probs_point=False,zoom=15,trail=rescue_plan,
         trail_enumerate=False,plot_origin=False)


In [ ]:
show_map(hiker,probs=rescue_grid,probs_point=False)

# Conclusion

In summary we have looked into Python function closure's and how to use them to avoid nested loop. We used a function closure to store information about the list size of each for loop, allowing us to create a 1D linear indexing that can be iterated with a single for loop instead. This function closure can  then be used to perform parameter optimization either through systematic grid search or random search iterations.

This approach allows for more readable and extensible code when compared with nested for loops.


